In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper, CategoricalImputer
from sklearn.impute import SimpleImputer
from sklearn.base import TransformerMixin


Funciones auxiliares

In [2]:
def leer_csv(ruta):
    return pd.read_csv(ruta, dtype={
    'tipodepropiedad':'category', 'ciudad':'category',\
    'provincia':'category'}, parse_dates=[16])

# funcion para setear los superficies
def set_metros(row):
    total = row.metrostotales
    covered = row.metroscubiertos
    if np.isnan(total):
        row.metrostotales = covered
        return row
    if np.isnan(covered):
        row.metroscubiertos = total
        return row
    return row

# Preprocessing 

Imputing Functions

In [3]:
# define numerical imputer
num_imputer = SimpleImputer(strategy='mean')

class SeriesImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        If the Series is of dtype Category, then impute with the most frequent object.
        """
    def fit(self, X, y=None):
        self.fill = X.value_counts().index[0]
        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

# define categorical imputer
cat_imputer = SeriesImputer()

def fill_valores(df, zeros, num_cols, cat_cols):
    if zeros:
        df[zeros] = df[zeros].fillna(value=0)
    if num_cols:
        df[num_cols] = num_imputer.fit_transform(df[num_cols])
    if cat_cols:
        for col in cat_cols:
            df[col] = cat_imputer.fit_transform(df[col])
    return df

Encoding functions

In [4]:
def one_hot_encoder(df, columnas):
    for col in columnas:
        df = pd.get_dummies(df, prefix = ['cat'], columns = [col])
    return df

Procesamiento de valores nulos

In [5]:
def processing_nulos(df, zeros_cols, num_cols, cat_cols, eliminar_sin_tipo=False):
    
    # Metros cubiertos y totales
    
    df[['metrostotales', 'metroscubiertos']] = \
    df[['metrostotales', 'metroscubiertos']].apply(set_metros, axis = 1)
    
    # Imputando otras columnas
    
    df = fill_valores(df, zeros_cols, num_cols, cat_cols)
    
    return df

Procesamiento de Fechas

In [6]:
def processing_fechas(df, anio, mes, dia):
    df['fecha'] = pd.to_datetime(df['fecha'])
        
    if anio:
        df['anio'] = df['fecha'].dt.year
    if mes:
        df['mes'] = df['fecha'].dt.month
    if dia:
        df['dia'] = df['fecha'].dt.day
    return df

Función suma_columnas

In [7]:
def suma_columnas(df, cols):
    nombre = ''
    for col in cols:
        nombre = nombre + col
    df['suma_'+nombre] = sum([df[col] for col in cols])
    return df

_____
### Funcion de preprocessing

In [8]:
def cambiar_valores(x):
    i = 0
    if x.name == 'precioporm2':
        i+=1
        return i
    return x

def encoder_idzona(df_train, df_test): 
    
    df_train['precioporm2'] = df_train['precio'] / df_train['metrostotales']

    precio_m2_por_idzona = df_train.groupby('idzona').agg({'precioporm2' : 'mean'})
    precio_m2_por_idzona.reset_index(inplace = True)
    precio_m2_por_idzona = precio_m2_por_idzona.sort_values(by = 'precioporm2')

    precio_m2_por_idzona = precio_m2_por_idzona.apply(cambiar_valores)
    precio_m2_por_idzona['precioporm2'] = precio_m2_por_idzona['precioporm2'].cumsum()

    orden_idzona = precio_m2_por_idzona.set_index('idzona').to_dict()
    orden_idzona = orden_idzona['precioporm2']

    df_train['idzona_ordinal'] = cat_imputer.fit_transform(df_train.idzona.map(orden_idzona))
    df_test['idzona_ordinal'] = cat_imputer.fit_transform(df_test.idzona.map(orden_idzona))
    
    return df_train, df_test


In [9]:
def encoder_ciudad(df_train, df_test):
    #filtrando los precios
    #hay que eliminar los nulls primero
    df_train['precioporm2'] = df_train['precio'] / df_train['metrostotales']
    df_train_ciudad = df_train.groupby('ciudad').agg({'precioporm2':'mean'}).sort_values('precioporm2').reset_index()
    df_train_ciudad['precioporm2'] = np.log(df_train_ciudad['precioporm2'])
    ciudad_log_precio = df_train_ciudad.set_index('ciudad').to_dict()['precioporm2']

    df_test['ciudad_ordinal'] = cat_imputer.fit_transform(df_test.ciudad.map(ciudad_log_precio))
    
    df_train['ciudad_ordinal'] = cat_imputer.fit_transform(df_train.ciudad.map(ciudad_log_precio))
    
    return df_train, df_test

In [10]:
def preprocessing(guardar_csv=False):
    df_train = leer_csv('data/train.csv')
    df_test  = leer_csv('data/test.csv')
    
    df_train = df_train[(df_train['tipodepropiedad'] != 'Hospedaje')&\
                        (df_train['tipodepropiedad'] != 'Garage')]
    
    ## Fechas
    
    df_train = processing_fechas(df_train, True, True, True)
    df_test = processing_fechas(df_test, True, True, True)
    
    ## Nulos
    
    zeros_cols_train = None
    num_cols_train = ['antiguedad', 'habitaciones', 'banos', 'garages']
    cat_cols_train = ['ciudad', 'tipodepropiedad', 'idzona']
    
    zeros_cols_test = None
    num_cols_test = ['antiguedad', 'habitaciones', 'banos', 'garages']
    cat_cols_test = ['ciudad', 'tipodepropiedad', 'idzona']
    
    df_train = processing_nulos(df_train, zeros_cols_train, num_cols_train, cat_cols_train, eliminar_sin_tipo=True)
    df_test = processing_nulos(df_test, zeros_cols_test, num_cols_test, cat_cols_test)
    
    ## Encoding de variables categóricas
    
    # One Hot encoding para 'tipo de propiedad'
    df_train['tipodepropiedad'].cat.remove_unused_categories()
    df_train = one_hot_encoder(df_train, ['tipodepropiedad'])
    df_test = one_hot_encoder(df_test, ['tipodepropiedad'])
    
    # Encoding ordinal de las ciudades
    
    df_train, df_test = encoder_ciudad(df_train, df_test)
    
    ## Otras Features
    
    # 'cant_extras' = 'gimnasio'+'usosmultiples'+'piscina'
    extras = ['gimnasio', 'usosmultiples', 'piscina']
    df_train = suma_columnas(df_train, extras)
    df_test = suma_columnas(df_test, extras)
    
    # 'cant_cercanos' = 'escuelascercanas'+'centroscomercialescercanos'
    cercanos = ['escuelascercanas', 'centroscomercialescercanos']
    df_train = suma_columnas(df_train, cercanos)
    df_test = suma_columnas(df_test, cercanos)
    
    
    ## Eliminamos columnas no utilizadas
    df_train.drop(columns=['fecha', 'id', 'titulo', 'descripcion','direccion',\
                           'lat', 'lng', 'provincia','ciudad','gimnasio','usosmultiples',\
                           'piscina','escuelascercanas','centroscomercialescercanos',\
                           'metroscubiertos','precioporm2', 'cat_Garage', 'cat_Hospedaje'], inplace=True)
    df_test.drop(columns=['fecha', 'id', 'titulo', 'descripcion', 'direccion',\
                          'lat', 'lng','provincia', 'ciudad','gimnasio','usosmultiples',\
                          'piscina','escuelascercanas','centroscomercialescercanos',\
                          'metroscubiertos'], inplace=True)
    
    ## Agregamos columna con log(precio)
    df_train['log_precio'] = np.log(df_train['precio'])
    
    if guardar_csv:
        df_train.to_csv('data/train_preproc.csv', index = False)
        df_test.to_csv('data/test_preproc.csv', index = False)
    return df_train, df_test

In [11]:
df_train, df_test = preprocessing(guardar_csv=True)

In [12]:
# tiene 2 columnas más: precio y log_precio
df_train.columns.nunique()

36

In [13]:
df_test.columns.nunique()

34

In [14]:
df_test.isnull().sum()

antiguedad                                         0
habitaciones                                       0
garages                                            0
banos                                              0
metrostotales                                      0
idzona                                             0
anio                                               0
mes                                                0
dia                                                0
cat_Apartamento                                    0
cat_Bodega comercial                               0
cat_Casa                                           0
cat_Casa en condominio                             0
cat_Casa uso de suelo                              0
cat_Departamento Compartido                        0
cat_Duplex                                         0
cat_Edificio                                       0
cat_Inmuebles productivos urbanos                  0
cat_Local Comercial                           

In [15]:
df_train.isnull().sum()

antiguedad                                         0
habitaciones                                       0
garages                                            0
banos                                              0
metrostotales                                      0
idzona                                             0
precio                                             0
anio                                               0
mes                                                0
dia                                                0
cat_Apartamento                                    0
cat_Bodega comercial                               0
cat_Casa                                           0
cat_Casa en condominio                             0
cat_Casa uso de suelo                              0
cat_Departamento Compartido                        0
cat_Duplex                                         0
cat_Edificio                                       0
cat_Huerta                                    

In [16]:
df_train.head()

,antiguedad,habitaciones,garages,banos,metrostotales,idzona,precio,anio,mes,dia,...,cat_Quinta Vacacional,cat_Rancho,cat_Terreno,cat_Terreno comercial,cat_Terreno industrial,cat_Villa,ciudad_ordinal,suma_gimnasiousosmultiplespiscina,suma_escuelascercanascentroscomercialescercanos,log_precio
0,8.116273,2.0,1.0,2.0,80.0,23533.0,2273000.0,2015,8,23,...,0,0,0,0,0,0,10.236050,0.0,0.0,14.636611
1,10.000000,3.0,2.0,2.0,180.0,24514.0,3600000.0,2013,6,28,...,0,0,0,0,0,0,10.031497,0.0,2.0,15.096444
2,5.000000,3.0,2.0,2.0,166.0,48551.0,1200000.0,2015,10,17,...,0,0,0,0,0,0,9.014225,0.0,0.0,13.997832
3,1.000000,2.0,1.0,1.0,67.0,53666.0,650000.0,2012,3,9,...,0,0,0,0,0,0,9.085374,0.0,2.0,13.384728
4,10.000000,2.0,1.0,1.0,95.0,47835.0,1150000.0,2016,6,7,...,0,0,0,0,0,0,9.647836,0.0,0.0,13.955273
